### Install Flask package using pip
The exclamation mark '!' is used to execute Linux commands. 
If the Flask package is installed, you can ignore the cell below.

In [335]:
#!pip3 install flask

In [336]:
#!pip install --upgrade flask


In [337]:
#!pip install watchdog

In [338]:
import os
import sqlite3
from datetime import datetime
from werkzeug.utils import secure_filename
from flask import Flask, render_template, request, jsonify

In [339]:
app = Flask(__name__)

In [340]:
# 資料庫路徑
DATABASE_FOLDER = os.path.join(app.root_path, 'instance')
UPLOAD_FOLDER = os.path.join(app.root_path, 'uploads')

In [341]:
# 設定允許的檔案類型
ALLOWED_EXTENSIONS = {'wav', 'mp3'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [342]:
# 確保所有需要的文件夾存在
for folder in [DATABASE_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [343]:
DATABASE = os.path.join(DATABASE_FOLDER, 'database.db')

In [344]:
# 定義根目錄
@app.route('/')
def index():
    return render_template('index.html')

In [345]:
# 定義分類目錄入口
@app.route('/classify')
def classify():
    return render_template('class.html')

In [346]:
#定義身體頁面
@app.route('/body')
def bodyPage():
   return render_template('classTemplate.html', title = "BODY")

In [347]:
#定義心靈頁面
@app.route('/psycho')
def psychoPage():
   return render_template('classTemplate.html', title = "PSYCHO")

In [348]:
#定義社會頁面
@app.route('/social')
def socialPage():
   return render_template('classTemplate.html', title = "SOCIAL")

In [349]:
#定義特殊頁面
@app.route('/special')
def specialPage():
   return render_template('classTemplate.html', title = "SPECIAL")

In [350]:
#定義其他頁面
@app.route('/extra')
def extraPage():
   return render_template('classTemplate.html', title = "EXTRA")

In [351]:
# 查詢姓名列表
@app.route('/fetchNameList', methods=['GET']) 
def fetchNameList():
    try:
        # 使用 with 語句管理資料庫連線
        with sqlite3.connect(DATABASE) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM nameList")  # 假設表格名稱為 'nameList'，字段名稱為 'name'
            names = [row[0] for row in cursor.fetchall()]
        return jsonify(names)

    except Exception as e:
        return jsonify({'error': str(e)}), 500


In [352]:
# 新增姓名到資料庫
@app.route('/addName', methods=['POST'])
def addName():
    try:
        data = request.get_json()
        new_name = data.get('name')
        if not new_name:
            return jsonify({'error': 'No name provided'}), 400
        
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO nameList (name) VALUES (?)", (new_name,))
        conn.commit()
        conn.close()
        return jsonify({'success': True}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [353]:
# 查詢指定人名的逐字稿
@app.route('/fetchTranscripts', methods=['GET'])
def fetchTranscripts():
    person = request.args.get('person')  # 從請求中獲取人名參數
    if not person:
        return jsonify({'error': 'No person provided'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        # 根據名稱查詢逐字稿
        cursor.execute("SELECT content, timestamp FROM transcripts WHERE name = ? ORDER BY timestamp DESC", (person,))
        rows = cursor.fetchall()
        transcripts = [{'content': row[0], 'timestamp': row[1]} for row in rows]
        conn.close()
        app.logger.info(f'Transcripts fetched for {person}: {transcripts}')

        return jsonify(transcripts)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [354]:
# 新增指定人名的逐字稿
@app.route('/uploadTranscript', methods=['POST'])
def uploadTranscript():
    data = request.get_json()
    name = data.get('name')
    content = data.get('content')
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    if not name or not content:
        app.logger.error('Name or content is missing in the request')
        return jsonify({'error': 'Name or content is missing'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO transcripts (name, content, timestamp) VALUES (?, ?, ?)", (name, content, timestamp))
        conn.commit()
        conn.close()
        app.logger.info(f'Transcript uploaded for {name} at {timestamp}')
        return jsonify({'message': 'Transcript uploaded successfully'}), 200
    except Exception as e:
        app.logger.error(f'Error uploading transcript for {name}: {e}')
        return jsonify({'error': str(e)}), 500

In [355]:
# 編輯指定人名的逐字稿
@app.route('/editTranscript', methods=['POST'])
def editTranscript():
    data = request.get_json()
    timestamp = data.get('timestamp')
    new_content = data.get('newContent')

    if not timestamp or not new_content:
        app.logger.error('Timestamp or new content is missing in the request')
        return jsonify({'error': 'Timestamp or new content is missing'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("UPDATE transcripts SET content = ? WHERE timestamp = ?", (new_content, timestamp))
        conn.commit()
        conn.close()
        app.logger.info(f'Transcript edited at {timestamp}')
        return jsonify({'message': 'Transcript edited successfully'}), 200
    except Exception as e:
        app.logger.error(f'Error editing transcript at {timestamp}: {e}')
        return jsonify({'error': str(e)}), 500

In [356]:
# 刪除指定人名的逐字稿
@app.route('/deleteTranscript', methods=['POST'])
def deleteTranscript():
    data = request.get_json()
    timestamp = data.get('timestamp')

    if not timestamp:
        app.logger.error('Timestamp is missing in the request')
        return jsonify({'error': 'Timestamp is missing'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("DELETE FROM transcripts WHERE timestamp = ?", (timestamp,))
        conn.commit()
        conn.close()
        app.logger.info(f'Transcript deleted at {timestamp}')
        return jsonify({'message': 'Transcript deleted successfully'}), 200
    except Exception as e:
        app.logger.error(f'Error deleting transcript at {timestamp}: {e}')
        return jsonify({'error': str(e)}), 500

In [357]:
# 上傳錄音檔
@app.route('/uploadRecord', methods=['POST'])
def upload_record():
    print("接收到上傳請求")
    if 'file' not in request.files:
        print("沒有檔案被上傳")
        return jsonify({'success': False, 'message': '沒有檔案被上傳'}), 400
    
    file = request.files['file']
    print(f"接收到的文件名稱：{file.filename}")

    if file.filename == '':
        print("文件名稱為空")
        return jsonify({'error': 'No selected file'}), 400

    # 檢查文件是否符合允許的類型
    if not allowed_file(file.filename):
        print(f"不允許的文件類型：{file.filename}")
        return jsonify({'error': 'File type not allowed'}), 400

    # 儲存上傳的檔案
    upload_folder = './uploads'
    if not os.path.exists(upload_folder):
        os.makedirs(upload_folder)

    file_path = os.path.join(upload_folder, file.filename)
    file.save(file_path)
    print(f"文件已成功保存到：{file_path}")

    return jsonify({'message': 'File uploaded successfully'}), 200


In [358]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.104:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Nov/2024 17:21:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:21:13] "GET /static/images/profile/01.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2024 17:21:13] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2024 17:21:13] "GET /static/js/index.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:21:13] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:21:14] "GET /WebExplain.vue.map HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2024 17:21:14] "GET /WebSummary.vue.map HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2024 17:21:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:21:19] "POST /uploadRecord HTTP/1.1" 400 -


接收到上傳請求
接收到的文件名稱：requirements.txt
不允許的文件類型：requirements.txt


127.0.0.1 - - [28/Nov/2024 17:23:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:23:20] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2024 17:23:20] "GET /static/images/profile/01.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2024 17:23:20] "GET /static/js/index.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:23:20] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:23:20] "GET /WebExplain.vue.map HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2024 17:23:20] "GET /WebSummary.vue.map HTTP/1.1" 404 -
127.0.0.1 - - [28/Nov/2024 17:23:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 17:23:25] "POST /uploadRecord HTTP/1.1" 400 -


接收到上傳請求
接收到的文件名稱：requirements.txt
不允許的文件類型：requirements.txt


127.0.0.1 - - [28/Nov/2024 17:23:34] "POST /uploadRecord HTTP/1.1" 200 -


接收到上傳請求
接收到的文件名稱：your-voice-file.wav
文件已成功保存到：./uploads\your-voice-file.wav
